# BM25 检索模型
## 流程
- 数据预处理
- 建立到排索引
- 实现OR操作
- 实现f(q,d）核函数
- 查询操作返回topK relevance 结果

In [100]:
import json
import nltk
import math
from bs4 import BeautifulSoup
import string

### 数据预处理
- 解析json格式数据得到dict_list
- 去非关键性标点，换行符等，split
- 建立vocab（Tokenize）
- 保存结果数据
###### 注意这里的 text texts的顺序相同 vbocab 一个文档只记录一次

In [108]:
f = open('./data/tweets.txt','r+')
lines = f.readlines()
text = []
docid = []
for l in lines:
    text.append(json.loads(l)['text'])
    docid.append(json.loads(l)['tweetId'])

In [111]:
texts = []
symbol = [',',':','_','!','\"','*','>','<','@','~','-','(',')','%','=','\\','^'
          ,'&','|','#','$','[',']','+',':','#','|'] 
for l in text:
    trantab = str.maketrans({key: None for key in string.punctuation})
    j = l.translate(trantab)
    j = j.split()
    texts.append(j)

### 建立vocab

In [28]:
vocab = {}
for line in texts:
    line_set = set(line) ##去重
    for word in line_set:       
            if vocab.get(word) == None:
                vocab[word] = 1
            else :
                vocab[word] += 1
v_tuple = sorted(vocab.items(),key = lambda x:x[1],reverse=True) ##从大到小排序
vocab = {}
for t in v_tuple:
    vocab[t[0]] = t[1]

In [29]:
f = open('./data/vocab.txt','w+')
line_str = json.dumps(vocab)
f.write(line_str)
f.close()

## 建立倒排索引

In [30]:
word2inverted_index = {} ##一个字典 存frequency 一个字典存 每个词出现的docID
keys = list(vocab.keys())
for k in keys:
    word2inverted_index[k] = []##初始化

for i,line in enumerate(texts):
    line_set = set(line) ##去重
    for word in line_set:
        word2inverted_index[word].append(i)

### 建立IDF字典

In [31]:
d_idf = [len(word2inverted_index[k]) for k in keys]

### 计算avgD

In [32]:
lend = [len(t) for t in texts]
avgD = sum(lend)/len(lend)

### OR操作

In [33]:
def OR_op(l1,l2):
    ##两个有序链表 返回并集 docID 并且去重 有序
    result = []
    i = 0
    j = 0
    while(i<len(l1) and j < len(l2)):
        if l1[i] == l2[j]:
            result.append(l1[i])
            
            i += 1
            j += 1
        else:
            if l1[i] < l2[j]:
                result.append(l1[i])
                i += 1
            else:
                result.append(l2[j])
                j += 1
    result.extend(l1[i:])
    result.extend(l2[j:])
    ##注意这里注意剩余
    return result

## 实现计算relevance 的核函数f(q,d)
- q:列表 query
- d:列表 document


In [291]:
ki = 5
b = 0.5
M = len(texts)
def count(w,d):
    i = 0
    for j in d:
        if  w== j:
            i+=1
    return i

def fqd(q,d):
    result_relevance  = 0
    qs = set(q)#去重
    middel = ki * (1-b + b*len(d)/avgD)
    for w in qs:
        if w in d:
            result_relevance += count(w,q)*(ki+1)*count(w,d)/(count(w,d)+middel)*math.log2((M+1.0)/d_idf[keys.index('House')])
    return result_relevance

### 实现topk查询

In [292]:
f_stop = open('stopword','r+')
stopwords = f_stop.readlines()
stopwords = [s.strip('\n') for s in stopwords]
f_stop.close()

def query_topk(Q,k = 20):
    
    trantab = str.maketrans({key: None for key in string.punctuation})
    j = Q.translate(trantab)
    j = j.split()
    qs = []
    for qqq in j:
        if qqq not in stopwords:
            qs.append(qqq)
        
    ls = [word2inverted_index[w]  for w in qs if w in keys]
    ds = []
    for l in ls:
        ds = OR_op(ds,l)
    relevace = {}## d - > relevance
    for d in ds:
        relevace[d] = fqd(qs,texts[d])
    
    v_tuple = sorted(relevace.items(),key = lambda x:x[1],reverse=True) ##从大到小排序
    relevace = {}
    for t in v_tuple:
        relevace[t[0]] = t[1]
    kk = list(relevace.keys())
    k = min(k,len(kk))
    return [texts[kk[i]] for i in range(k)]

def query_topk_id(Q,k = 20):
    
    trantab = str.maketrans({key: None for key in string.punctuation})
    j = Q.translate(trantab)
    j = j.split()
    qs = []
    for qqq in j:
        if qqq not in stopwords:
            qs.append(qqq)
    
    ls = [word2inverted_index[w]  for w in qs if w in keys]
    ds = []
    for l in ls:
        ds = OR_op(ds,l)
    relevace = {}## d - > relevance
    for d in ds:
        relevace[d] = fqd(qs,texts[d])
    
    v_tuple = sorted(relevace.items(),key = lambda x:x[1],reverse=True) ##从大到小排序
    relevace = {}
    for t in v_tuple:
        relevace[t[0]] = t[1]
    kk = list(relevace.keys())
    k = min(k,len(kk))
    return [docid[kk[i]] for i in range(k)]   

In [293]:
def inred( s ):
    return"%s[31;2m%s%s[0m"%(chr(27), s, chr(27))
def print_line_with_important(line , Q):
    trantab = str.maketrans({key: None for key in string.punctuation})
    j = Q.translate(trantab)
    j = j.split()
    s_line = j
    to_be_print = ''
    for l in line:
        if l in s_line:
            to_be_print += inred(l) +' ' ##这里强调
        else :
            to_be_print += l + ' '
    print(to_be_print)

def print_top_k(Q):
    for line in query_topk(Q):
        print_line_with_important(line,Q)

def save_top_k(Qid,Q,k,f):
    for l in query_topk_id(Q,k):
        f.write(str(Qid)+" "+str(l)+"\n") 

In [295]:
f = open('query.txt','r+')
Soup = BeautifulSoup(f,'lxml'); 
query = Soup.select('top > query')
query = [q.getText() for q in query]
queryid = Soup.select('top > num')
queryid = [q.getText()[11:-1] for q in queryid]
f.close()

In [296]:
f = open('result.txt','w+')
for i,q in  enumerate(query):
    save_top_k(queryid[i],q,200,f)
f.close()

In [303]:
print_top_k(query[4])
query[4]

I am naming this storm Finding Nemo wheresheat 
They are naming snow storms now Winter storm Nemo Really 
Did they name the storm Nemo so every news and weather outlet can use the phrase Finding Nemo when tracking the storm Nemo 
NormansCat Nemo They named the storm Nemo Wow awesome 
When the hell did we start naming nonhurricanes Storm Nemo Isnt that a cartoon fish How scary is a storm named Nemo 
Wait they named this storm Nemo As in Finding Nemo 
Why would they name this storm Nemo Whats next super storm WallE 
Winter storm Nemo httptcoSVlr8SXr 
Winter storm Nemo Who gives names to these storms and when did they start naming winter storms 
httptcoheJAhFSp is kind of undercutting their terrifying YOU MUST PREPARE NOW headlines by naming the storm Nemo 
Lol Winter storm Nemo lol 
Dont call this storm Nemo or Nemo or you let the Weather Channel win httptco9oBu4eQX 
Who names the worst winter storm ever Nemo Why not Godzilla How about not naming winter storms tcot NRA 
Since when did th

' commentary on naming storm Nemo '

### 对结果进行evaluate

In [297]:
from eval_hw4.eval_hw4 import *

#### 我的结果


In [298]:
k = 200
# query relevance file
file_qrels_path = 'eval_hw4/qrels.txt'
# qrels_dict = {query_id:{doc_id:gain, doc_id:gain, ...}, ...}
qrels_dict = generate_tweetid_gain(file_qrels_path)
# ur result, format is in function read_tweetid_test, or u can write by ur own
file_test_path = 'result.txt'
# test_dict = {query_id:[doc_id, doc_id, ...], ...}
test_dict = read_tweetid_test(file_test_path)
MAP = MAP_eval(qrels_dict, test_dict, k)
print('MAP', ' = ', MAP, sep='')
NDCG = NDCG_eval(qrels_dict, test_dict, k)
print('NDCG', ' = ', NDCG, sep='')

query: 171 ,AP: 0.74986098241
query: 172 ,AP: 0.590818781865
query: 173 ,AP: 0.118333333333
query: 174 ,AP: 0.376227284106
query: 175 ,AP: 0.661437817496
query: 176 ,AP: 0.626031701265
query: 177 ,AP: 0.649910382997
query: 178 ,AP: 0.863223572563
query: 179 ,AP: 0.478993432505
query: 180 ,AP: 0.289921998097
query: 181 ,AP: 0.888888888889
query: 182 ,AP: 0.3861003861
query: 183 ,AP: 0.811003459983
query: 184 ,AP: 0.678947573203
query: 185 ,AP: 0.647339998585
query: 186 ,AP: 0.77135739991
query: 187 ,AP: 0.85291125291
query: 188 ,AP: 0.429979693289
query: 189 ,AP: 0.0255610290093
query: 190 ,AP: 0.452936289754
query: 191 ,AP: 0.547970814244
query: 192 ,AP: 0.444852892877
query: 193 ,AP: 0.5764088747
query: 194 ,AP: 0.32293675667
query: 195 ,AP: 0.418122673981
query: 196 ,AP: 0.667297050938
query: 197 ,AP: 0.79233487683
query: 198 ,AP: 0.538541915733
query: 199 ,AP: 0.47204961295
query: 200 ,AP: 0.691697952413
query: 201 ,AP: 0.732614801206
query: 202 ,AP: 0.794842533135
query: 203 ,AP: 0

#### 对比结果

In [299]:
k = 200
# query relevance file
file_qrels_path = 'eval_hw4/qrels.txt'
# qrels_dict = {query_id:{doc_id:gain, doc_id:gain, ...}, ...}
qrels_dict = generate_tweetid_gain(file_qrels_path)
# ur result, format is in function read_tweetid_test, or u can write by ur own
file_test_path = 'eval_hw4/hisresult.txt'
# test_dict = {query_id:[doc_id, doc_id, ...], ...}
test_dict = read_tweetid_test(file_test_path)
MAP = MAP_eval(qrels_dict, test_dict, k)
print('MAP', ' = ', MAP, sep='')
NDCG = NDCG_eval(qrels_dict, test_dict, k)
print('NDCG', ' = ', NDCG, sep='')

query: 171 ,AP: 0.94980405976
query: 172 ,AP: 0.682593856655
query: 173 ,AP: 0.997813620072
query: 174 ,AP: 0.567534780035
query: 175 ,AP: 0.778210116732
query: 176 ,AP: 0.827412933877
query: 177 ,AP: 0.465198145065
query: 178 ,AP: 0.915379647212
query: 179 ,AP: 0.971163259061
query: 180 ,AP: 0.220973360037
query: 181 ,AP: 1.0
query: 182 ,AP: 0.3861003861
query: 183 ,AP: 0.851063829787
query: 184 ,AP: 0.953864220785
query: 185 ,AP: 0.565475418125
query: 186 ,AP: 0.986660079051
query: 187 ,AP: 0.909298308663
query: 188 ,AP: 0.818160705003
query: 189 ,AP: 0.37571536147
query: 190 ,AP: 0.93514660103
query: 191 ,AP: 0.878837663343
query: 192 ,AP: 1.0
query: 193 ,AP: 1.0
query: 194 ,AP: 0.977071661998
query: 195 ,AP: 0.515391404296
query: 196 ,AP: 1.0
query: 197 ,AP: 0.998919203817
query: 198 ,AP: 1.0
query: 199 ,AP: 0.475059382423
query: 200 ,AP: 0.712377070122
query: 201 ,AP: 0.740740740741
query: 202 ,AP: 0.924676840984
query: 203 ,AP: 0.713699248416
query: 204 ,AP: 0.899091826819
query: